# Image Classification on Cats vs. Dogs with Pretrained NN

In this notebook, we work with the pre-trained DenseNet network [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html) to classify cat and dog photos (see previous notebook).

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries,-load-and-transform-data" data-toc-modified-id="Import-libraries,-load-and-transform-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries, load and transform data</a></span></li><li><span><a href="#Transfer-Learning" data-toc-modified-id="Transfer-Learning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Transfer Learning</a></span><ul class="toc-item"><li><span><a href="#Load-pretrained-network" data-toc-modified-id="Load-pretrained-network-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load pretrained network</a></span></li><li><span><a href="#Freeze-parameters-and-replace-final-layer(s)" data-toc-modified-id="Freeze-parameters-and-replace-final-layer(s)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Freeze parameters and replace final layer(s)</a></span></li></ul></li><li><span><a href="#Train-The-Network-on-GPU" data-toc-modified-id="Train-The-Network-on-GPU-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Train The Network on GPU</a></span></li></ul></div>

## Import libraries, load and transform data

In [1]:
from collections import OrderedDict

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [2]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225]),
                                      ])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225]),
                                     ])

# Pass transforms to get the data
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

## Transfer Learning
### Load pretrained network

In [3]:
model = models.densenet121(pretrained=True)

# Check architecture
model

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:00<00:00, 48485883.74it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

### Freeze parameters and replace final layer(s)

In [4]:
# Freeze parameters so we don't backprop through them - we want them unaltered
for param in model.parameters():
    param.requires_grad = False

# Define new classifier layer(s) matching our problem
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

# Replace the output layer
model.classifier = classifier

## Train The Network on GPU

In [6]:
"""Version with high verbosity"""

# Write device agnostic code which will automatically use CUDA if it's enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)  # only train fc parameters!
model = model.to(device)

epochs = 10
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
        steps +=1
        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        log_probs = model.forward(inputs)
        loss = criterion(log_probs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    log_probs = model.forward(inputs)
                    batch_loss = criterion(log_probs, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    probs = torch.exp(log_probs)
                    top_p, top_class = probs.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            
            running_loss = 0
            # turn training mode / Dropout on again
            model.train() 

Device: cuda:0
Epoch 1/10.. Train loss: 1.580.. Test loss: 1.354.. Test accuracy: 0.488
Epoch 1/10.. Train loss: 0.908.. Test loss: 0.966.. Test accuracy: 0.527
Epoch 1/10.. Train loss: 1.078.. Test loss: 0.128.. Test accuracy: 0.959
Epoch 1/10.. Train loss: 2.033.. Test loss: 3.455.. Test accuracy: 0.488
Epoch 1/10.. Train loss: 3.123.. Test loss: 0.289.. Test accuracy: 0.883
Epoch 1/10.. Train loss: 0.393.. Test loss: 0.711.. Test accuracy: 0.804
Epoch 1/10.. Train loss: 1.589.. Test loss: 1.649.. Test accuracy: 0.683
Epoch 1/10.. Train loss: 2.179.. Test loss: 0.469.. Test accuracy: 0.916
Epoch 1/10.. Train loss: 0.722.. Test loss: 0.375.. Test accuracy: 0.926
Epoch 1/10.. Train loss: 2.714.. Test loss: 3.687.. Test accuracy: 0.695
Epoch 1/10.. Train loss: 7.654.. Test loss: 4.124.. Test accuracy: 0.680
Epoch 1/10.. Train loss: 4.199.. Test loss: 0.405.. Test accuracy: 0.933
Epoch 1/10.. Train loss: 0.826.. Test loss: 1.802.. Test accuracy: 0.834
Epoch 1/10.. Train loss: 9.006.. Tes

KeyboardInterrupt: 

In [ ]:
"""Alternative, lower verbosity, allows to visualize losses after training (see Notebook 5)"""

# Write device agnostic code which will automatically use CUDA if it's enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)  # only train fc parameters!
model = model.to(device)

epochs = 10

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        log_probs = model.forward(inputs)
        loss = criterion(log_probs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Implement the validation pass and print out the validation accuracy
        with torch.no_grad():
            model.eval()  # turn off Dropout / turn on inference mode
            for inputs, labels in testloader:
                # Move input and label tensors to the GPU
                inputs, labels = inputs.to(device), labels.to(device)
                
                log_probs = model(inputs)
                test_loss += criterion(log_probs, labels)
                probs = torch.exp(log_probs)
                top_p, top_class = probs.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        # turn training mode / Dropout on again
        model.train() 
        
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))
        
        print(f'Epoch: {e+1}/{epochs}',
              f'Training Loss: {running_loss/len(trainloader):.3f}',
              f'Test Loss: {test_loss/len(testloader):.3f}',
              f'Accuracy: {accuracy/len(testloader):.3f}')

---